In [ ]:
!pip install pandas
import pandas as pd

# Replace 'your_file_name.csv' with the actual name of your CSV file
# If the CSV is not in the same directory as your notebook,
# you'll need to provide the full or relative path.
df = pd.read_csv('audible_uncleaned.csv')

# Display the first few rows of the DataFrame to confirm it loaded correctly
print(df.head())